## 瀏覽器&套件設定

In [2]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [3]:
# 擋掉通知黑屏，以防止瀏覽器第一次登入時跳出通知視窗
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

## 自動登入

In [4]:
# ------ 登入的帳號與密碼 ------
username = "帳號"
password="密碼"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

url = 'https://www.facebook.com'  
driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))

elem = driver.find_element(By.ID,"email")
elem.send_keys(username)

elem = driver.find_element(By.ID,"pass")
elem.send_keys(password)


# 提交登入
elem.send_keys(Keys.RETURN)
time.sleep(5)

# 檢查有沒有被擋下來
if len(driver.find_elements(By.XPATH,"//*[contains(text(), '你的帳號暫時被鎖住')]")) > 0:
    driver.find_elements(By.XPATH,"//*[contains(text(), '是')]")[1].click()
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), '用戶名稱或密碼無效')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), 'Invalid username or password')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
else:
    feed = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]'))) # 等待動態頁面(鎖定貼文)加載完成
    print("登入成功")

登入成功


## 純貼文爬取

In [13]:
posts_number = int(input("請輸入要爬取的貼文數量:"))
posts = []

driver.set_window_size(1000,4000)
# 初始設定
last_height = driver.execute_script("return document.body.scrollHeight") # 取得頁面高度初始值
last_count = 0 # 計算目前貼文數量

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="main"]') # 找到動態頁面(鎖定貼文)

In [14]:
while True:
    # 如果已經找到足夠的貼文，則停止
    if len(posts) >= posts_number:
        posts = posts[:posts_number]
        break
    
    # 貼文文字的 class selector
    post_text_class_selector = 'div[data-ad-preview="message"]'
    
    # 捕捉所有貼文元素
    
    elements = feed.find_elements(By.CSS_SELECTOR, 'div[class = x1lliihq]')
    current_count = len(elements)    
    if current_count == 0:
        elements = feed.find_elements(By.CSS_SELECTOR, '[data-pagelet]')
        current_count = len(elements)
    

    if current_count != last_count:
        try:
            print("!!! 檢查當前是否有 「查看更多」 Button !!!")
            buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), '查看更多') and @role='button']")))
            for button in buttons:

                # 滾動到按鈕位置
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                WebDriverWait(driver, 5).until(EC.visibility_of(button))
                # 使用 JavaScript 強制點擊按鈕
                driver.execute_script("arguments[0].click();", button)
                print('Button clicked!')
        except Exception as e:
            print(f"當前無「查看更多」 Button")
            pass

        print('!!! 檢查完畢 !!!')
        while len(posts) < posts_number and len(elements) > last_count:
            current_element = elements[last_count]
            last_count += 1
            try:
                print('--------------貼文分隔線--------------')
                author_text = current_element.find_element(By.CSS_SELECTOR, 'h4').text.split('\n')[0]
                element_post = current_element.find_element(By.CSS_SELECTOR, post_text_class_selector)
                post_text = element_post.text
                posts.append({'author': author_text, 'text': post_text})
                print(author_text)
                print(post_text)
            except Exception as e:
                # print(e)
                print('此篇非普通貼文，跳過處理')
        print('---------------------------------')

    # 計算新的滾動高度並比較是否到達底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    last_height = new_height

    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待新內容加載(秒)
    time.sleep(4)
driver.refresh()
print('共爬取', len(posts), '篇貼文')
print(posts)

!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
2000Fun遊戲
一部電腦最緊要係咩，係電
所以一隻穩定而又好嘅火牛係好緊要
Seasonic Focus 系列提供10年保養
Seasonic Vertex 系列提供12年保養
Seasonic Summer Promotion 開始喇
了解更多:
https://www.openshop.com.hk/app/special/723.html
#Seasonic #信心之選 #穩定 #安全 #ATX #SFX
歡迎whatsapp報價
FPS轉數快/支付寶/WECHAT PAY/信用卡 多種支付方式不收手續費
産品多元化,一站式既購物體驗
網上落單,足不出戶都可以買到心儀既電競/電腦産品
門市遍佈港九新界,等你唔洗出深水埗都可以享受到腦場價!
網上落單門市攞又得,門市落單送貨上門又得!
砌機仲有得免費送貨(偏遠地區除外)
#免費送貨 #砌機 #2000Fun #自選砌機
--------------貼文分隔線--------------
美麗佳人 Marie Claire Taiwan 
打造超越預期的晶瑩剔透！#謝謝PITERA 讓肌膚保持經典不敗的狀態！
日本國民女神永野芽郁從24歲開始用SK-II青春露，分享她的超乎預期的透明美肌心得
立即試用兌換>> https://bit.ly/3xA6f7L
還有與日本同價享受
新客組三千有找~還能現折300！還不快入手
#skiipartnership #SKIITW #晶瑩剔透美肌#PITERA透亮顏究社#謝謝PITERA
---------------------------------
!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
此篇非普通貼文，跳過處理
--------------貼文分隔線--------------
緯來日本台 
#緯日小編直擊
 　　日片《福田村事件》
◥◣#田中麗奈 & #井浦新 來台!◢◤
.
或許你們沒聽說過這部電影
對這個